# Create crop map for February 2021 to August 2021 season

In [1]:
# Load required python packages
import os

import numpy as np
import geopandas as gpd
from deafrica_tools.classification import predict_xr
from deafrica_tools.dask import create_local_dask_cluster
from feature_collection import feature_layers
from joblib import load
from odc.geo.geom import Geometry
from odc.geo.xr import write_cog, rasterize

In [ ]:
# Create a dask cluster for running predictions.
create_local_dask_cluster()

In [3]:
# Set the anlaysis parameters.

# This parameters should be the same as those used in the Extract Training features notebook.
output_crs = "EPSG:6933"
resolution = (-10, 10)
time_range = ("2021-02", "2021-08")
dask_chunks = dict(x=3200,y=3200)

# Path to the area of interest vector file
aoi_file = "data/aoi.geojson"

# Path to the model to use for prediction
output_folder = "results"
model_path = os.path.join(output_folder, "feb_2021_to_aug_2021_ml_model.joblib")

# Value to use to replace nan when writing predictions to disk.
no_data_value = -999

In [4]:
# Load the model
model = load(model_path).set_params(n_jobs=1)

In [5]:
# Load the area of interest
aoi_gdf = gpd.read_file(aoi_file).to_crs(output_crs)
aoi_geopolygon = Geometry(geom=aoi_gdf.iloc[0].geometry, crs=aoi_gdf.crs)

In [6]:
# Create a datacube query object using our analysis parameters.
query = dict(output_crs=output_crs, resolution=resolution, time=time_range, dask_chunks=dask_chunks, geopolygon=aoi_geopolygon)

In [7]:
%%time
# Load the feature data.
data = feature_layers(query)

CPU times: user 5.44 s, sys: 304 ms, total: 5.74 s
Wall time: 5.58 s


In [8]:
%%time
# Predict using the imported model
predicted = predict_xr(model=model, input_xr=data, proba=True, persist=False, clean=True, return_input=False).compute()

predicting...
   probabilities...


/usr/local/lib/python3.10/dist-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(


CPU times: user 1min 57s, sys: 12.2 s, total: 2min 10s
Wall time: 24min 18s


In [9]:
# Mask the image using the area of interest.
mask = rasterize(poly=aoi_geopolygon, how=predicted.odc.geobox)
predicted_masked = predicted.where(mask)

In [ ]:
predicted_masked.fillna(no_data_value)
predicted_masked.attrs['nodata'] = no_data_value
predicted_masked = predicted_masked.astype(np.int16)

In [10]:
# Write the prediction and probabilities images to disk.
write_cog(geo_im=predicted_masked.Predictions, fname=os.path.join(output_folder, "feb_2021_to_aug_2021_predictions.tif"), nodata=no_data_value, overwrite=True)
write_cog(geo_im=predicted_masked.Probabilities, fname=os.path.join(output_folder, "feb_2021_to_aug_2021_probabilities.tif"), nodata=no_data_value, overwrite=True)

PosixPath('results/feb_2021_to_aug_2021_probabilities.tif')